### why : To Create of batches of data 

In [5]:
import torch
import pandas as pd
import torch.nn as nn 
import torch.optim as opt
from sklearn.preprocessing import LabelEncoder , StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [6]:
# Solution - just use two loops one for batch and other for epoch
# Probelm with this solution - Transformation not possibe, shuffling and sampling not possible , paarllel 

# DataSet class
# __init__ --> from where the data is coming from
# __len__ --> give total number of rows
# __getitem__ --> return row of given index

# Data Loader class
# shuffle --> make chunks --> pack chunks and make batch --> traning pipeline

 

In [7]:
df = pd.read_csv("Dataset/Titanic/train.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
140,141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,NaN,0,2,2678,15.2458,NaN,C
190,191,1,2,"Pinsky, Mrs. (Rosa)",female,32.0,0,0,234604,13.0000,NaN,S
285,286,0,3,"Stankovic, Mr. Ivan",male,33.0,0,0,349239,8.6625,NaN,C
642,643,0,3,"Skoog, Miss. Margit Elizabeth",female,2.0,3,2,347088,27.9000,NaN,S
355,356,0,3,"Vanden Steen, Mr. Leo Peter",male,28.0,0,0,345783,9.5000,NaN,S


In [8]:
# Preprocss the datset
df = df.drop(columns= ['Name' , 'Cabin' ,"PassengerId" , "Ticket"])

# Split the dataset
y = df['Survived']
X = df.drop(columns= ['Survived'])
X_train , x_test , y_train , y_test = train_test_split(X, y , test_size= 0.2 , random_state= 42)
X_train.shape , x_test.shape

# impuation for Age
Si = SimpleImputer(strategy= "mean")
X_train['Age'] = Si.fit_transform(X_train[['Age']])
x_test['Age'] = Si.transform(x_test[["Age"]])

# Impuation for Embarked
X_train = X_train.fillna({'Embarked': 'missing'})  # Replace with 'missing' instead of dropping
x_test = x_test.fillna({'Embarked': 'missing'})

# OHE
object_columns = X_train.select_dtypes(include=['object']).columns.tolist()
ohe = OneHotEncoder(sparse_output= False)

X_train_ohe_encoder = ohe.fit_transform(X_train[object_columns])
x_test_ohe_encoder = ohe.transform(x_test[object_columns])

X_train_ohe_df = pd.DataFrame(X_train_ohe_encoder , columns= ohe.get_feature_names_out(object_columns), index= X_train.index)
x_test_ohe_df = pd.DataFrame(x_test_ohe_encoder , columns= ohe.get_feature_names_out(object_columns) , index= x_test.index)

X_train = pd.concat([X_train , X_train_ohe_df] , axis= 1)
x_test = pd.concat([x_test , x_test_ohe_df] , axis= 1)

X_train = X_train.drop(object_columns , axis= 1)
x_test = x_test.drop(object_columns , axis= 1)

# Scale Values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)

# Label Encoder
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_test = LE.transform(y_test)
X_train.shape , y_train.shape


((712, 11), (712,))

In [9]:
# Converting into tensor
import numpy as np 

X_train = np.array(X_train)
x_test = np.array(x_test)
X_train = torch.from_numpy(X_train)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)
y_train = torch.from_numpy(y_train)

In [17]:
from torch.utils.data import DataLoader , Dataset

class CustomDataSet(Dataset):
    def __init__(self, featues , labels):
        self.featues = featues
        self.labels = labels
        
    def __len__(self):
        return self.featues.shape[0]

    def __getitem__(self, index):
        return self.featues[index] , self.labels[index]

In [22]:
dataset = CustomDataSet(X_train , y_train)
dataloader = DataLoader(dataset , batch_size= 16 , shuffle= True)


(tensor([-0.4006, -0.5005, -0.4707, -0.4793, -0.3771, -0.7243,  0.7243, -0.4615,
         -0.3034,  0.5968, -0.0531], dtype=torch.float64),
 tensor(0))